## Intro
This notebook provides the code used to verify the results in the paper, "Applications of Conformal Geometric Algebra to Transmission Line Theory". It makes use of the [clifford](http://clifford.readthedocs.io) python module.

###  Setup

In [2]:
#%nbtoc
from clifford import * 
pretty(precision=2)

# create algebra 
layout, blades = Cl(3,1)

# assign vector basis
e1,e2,e3,e4 = [blades['e%i'%k] for k in range(1,5)]

# setup  null basis, and minkowski subspace bivector
eo = .5^(e4-e3)
einf= e3+e4
E0= einf^eo
I = e1^e2^e3^e4



# convenience funcs to create vectors
v4 =  lambda : randomMV(layout, grades=[1])
def v2():
    x=v4()
    return x- E0.project(x)

cot = lambda x: 1./tan(x)

# short rotation func for versers
Rot = lambda V,X: V*X*V.inv()
# translation generator 
T =lambda a: 1+(.5^(einf*a))

###  CGA and Complex-Vector Maps

In [3]:
# complex <-> vector 
c2v = lambda x: (x.real*e1) + (x.imag*e2) # complex2vector
v2c = lambda x: float(x|e1)+ float(x|e2)*1j

# CGA up/down projections
up = lambda x: x + (.5^((x**2)*einf)) + eo
homo = lambda x: x * (-x|einf).normalInv() # homogenise conformal vector
down = lambda x: (homo(x)^E0)*E0

In [4]:
def equivalent(f_c, F,  args=[], debug=False):
    '''
    Test for equivalence of a complex function and a CGA operator. 
     
    A complex function f, operates on a random number x_c 
        y_c = f(x_c)
        
    the result y_c is compared to the map
        x_c -> x -> X -> Y -> y -> y_c
        
    Where x,y are G2 vectors, X,Y are CGA vectors, and Y=V*X*~V
    '''
    tol = 1e-6
    x_c = rand()+ rand()*1j 
    x = c2v(x_c) 
    X = up(x)
    vargs = args[:]
    for k in range(len(vargs)):
        if imag(vargs[k])!= 0:
            vargs[k] = c2v(vargs[k])
    Y = F(X, *vargs)
    y = down(Y)
    y_c = v2c(y)
    if debug:
        print  (y_c, f_c(x_c,*args))
        return abs(y_c - f_c(x_c,*args)) < tol
    return abs(y_c - f_c(x_c,*args)) < tol

### Complex operations

In [5]:
##  Complex Conjugation ( Z<->Z~)
f_c = lambda x: x.conjugate()
F = lambda X:  e2*X*e2
assert equivalent(f_c,F)

##  Complex Inversion ( Z<->Y)
f_c = lambda x: 1./x
F = lambda X:  Rot(e2^e3,X)
assert equivalent(f_c,F)

# define this as a function in case we use it later
CI = lambda X: Rot((e2^e3),X) # complex inversion

### Basis Transformations

In [6]:
## basis transforms
Rzy = e**(-pi/2*(e2^e3))
Ryz = ~Rzy
Rsz = e**(pi/4*(e1^e3))
Rzs = ~Rsz
Rsy = e**(-pi/(sqrt(2)*2)*(e2*e3+e2*e1))
Rys = ~Rsy

assert(Rsz*Rzy ==Rsy  )

##  Basis Transform ( Z<->S)
f_c = lambda x: (x-1)/(x+1)
F = lambda X:  Rot(Rsz,X)
assert equivalent(f_c,F)


##  Basis Transform  ( Y<->S)
f_c = lambda x: (1-x)/(1+x)
F = lambda X:  Rot(Rsy,X)
assert equivalent(f_c,F)

## Basis Transform  ( Y<->Z)
# already tested above by  the complex inversion tests
f_c = lambda x: 1/x
F = lambda X:  Rot(Ryz,X)
assert equivalent(f_c,F)

###  Matched Transmission lines in S, Z, and Y

In [7]:
## Transmission lines
Ls = lambda theta: e**(theta *(e1^e2))
Lz = lambda theta: e**(-theta *(e2^e3))

## Transmission line  (S)
f_c = lambda x,theta: x*e**(-2j*theta)
F = lambda X,theta:  Rot(Ls(theta),X)
theta = rand()
assert equivalent(f_c,F, args =[theta])

## Transmission line  (Z) and (Y) 
f_c = lambda x,theta: (x+1j*tan(theta))/(1+x*1j*tan(theta))
F = lambda X,theta:  Rot(Lz(theta),X)
theta = rand()
assert equivalent(f_c,F, args =[theta])


### Distributed Element Group

In [8]:
# bivector algebra 
R =  e3*e4-e1*e3
X =  e1*e2-e2*e4
G =  e3*e4+e1*e3
B =  e1*e2+e2*e4
N =  e1*e4
Q = -e2*e3
A =  e3^e4
L =  e1*e2

half=.5
# Rotors
Rr = lambda x:e**(half*x*R)
Rx = lambda x:e**(half*x*X)
Rg = lambda x:e**(half*x*G)
Rb = lambda x:e**(half*x*B)
Rn = lambda x:e**(-half*log(x)*N)
Rq = lambda x:e**(half*x*Q)
Rl = lambda x:e**(x*L)
Ra = lambda x:e**(-half*log(x)*A)

#test rotors do what we want
x = rand()
assert(Rsz*T(x*e1)*Rzs == Rr(x))
assert(Rsz*T(x*e2)*Rzs == Rx(x))
assert(Rsy*T(x*e1)*Rys == Rg(x))
assert(Rsy*T(x*e2)*Rys == Rb(x))


#N and Q dilate and Rotate z
Rzs*N*Rsz == e3^e4
Rzs*Q*Rsz == e1^e2

## bivector table
assert(R.x(G)==-2*N)
assert(R.x(X)==0)
assert(R.x(B)==2*Q)
assert(R.x(N)==R)
assert(R.x(Q)==-X)

assert(G.x(X)==-2*Q)
assert(G.x(B)==0)
assert(G.x(N)==-G)
assert(G.x(Q)==B)

assert(X.x(B) ==2*N)
assert(X.x(N) ==X)
assert(X.x(Q) ==R)

assert(B.x(N)==-B)
assert(B.x(Q)==-G)

assert(N.x(Q) ==0)

# orthognality 
B|X == G|B == Q|N == 0

# duality relations 
assert(R*I == X)
assert(G*I == B)
assert(A*I == L)
assert(Q*I == N)

# time-reversal relation
assert( e4*X*e4 == -B)
assert( e4*R*e4 ==  G)
assert( e4*N*e4 ==  N)
assert( e4*Q*e4 == -Q)


# bivector classification
# light-like
assert(R**2==G**2==X**2==B**2==0)
#timelike
assert(N**2==A**2==1)
#spacelike
assert(Q**2==L**2==-1)


# intermediate functions used for verification
s2z = lambda x: (1+x)/(1-x)
z2s = lambda x: (x-1)/(x+1)
s2y = lambda x: 1./s2z(x)
y2s = lambda x: z2s(1./x)

## Resistance
f_c = lambda x,z: z2s(s2z(x)+z)
F = lambda A,a:  Rot(Rr(a),A)
z = rand()
assert equivalent(f_c,F, args =[z])

## Reactance
f_c = lambda x,z: z2s(s2z(x)+z*1j)
F = lambda A,a:  Rot(Rx(a),A)
z = rand()
assert equivalent(f_c,F, args =[z])

## Conductance
f_c = lambda x,z: y2s(s2y(x)+z)
F = lambda X,x:  Rot(Rg(x),X)
z = rand()
assert equivalent(f_c,F, args=[z])


## Susceptance
f_c = lambda x,z: y2s(s2y(x)+z*1j)
F = lambda X,x:  Rot(Rb(x),X)
z = rand()
assert equivalent(f_c,F, args=[z])


## Impedance-scaling
f_c = lambda x,n: z2s(s2z(x)*n)
F = lambda X,x:  Rot(Rn(x),X)
z = rand()
assert equivalent(f_c,F, args=[z])

## Impedance-rotating
f_c = lambda x,theta: z2s(s2z(x)*exp(1j*theta))
F = lambda X,x:  Rot(Rq(x),X)
z = rand()
assert equivalent(f_c,F, args=[z])

### Relating Distributed Elements to Impedance Step

In [17]:
x,b = rand(2)
l = 1
n = sqrt(x/b)
theta = l*sqrt(x*b)

assert(e**(l/2.*(x*X+b*B)) == Rn(n)*Rl(theta)*~Rn(n))

In [18]:
assert(down(Rn(n)*eo*~Rn(n)) == e1*(n-1)/(n+1))

### Stubs

In [11]:
tol=1e-9

#stubs
Rss = lambda theta: e**(-cot(theta)/2*B)

## Shunt short
f_c = lambda x,theta: (1-((1-x)/(1+x)-1j*cot(theta)))/(1+((1-x)/(1+x)-1j*cot(theta)))
F = lambda X,theta:  Rot(Rss(theta),X)
theta = rand()
assert equivalent(f_c,F, args=[theta])

## down-projected formula 
theta, phi = rand(2)
g = e**(-2j*theta)*(1j*cot(phi))/(2-1j*cot(phi)) # series then shunt stub
assert(abs(g)- (sqrt(1/(4*tan(phi)**2+1)))<tol)
assert(tan(angle(g)) - (tan(2*theta)+2*tan(phi))/(2*tan(2*theta)*tan(phi)-1)<tol)

### Impedance Transformer

In [12]:
Rml90 = lambda n: ~Rn(n)*Rl(pi/2)*Rn(n)
## Impedance-rotating
f_c = lambda x,z: 1/x*z**-2
F = lambda X,x: Rot(Rzs*Rml90(x)*Rsz,X)
z = rand()
assert equivalent(f_c,F, args=[z])

In [14]:
r,b,x,g = rand(4)
V = exp(r*R+b*B+g*G+x*X)
V**2

-0.28 + (1.09^e12) + (0.0^e13) - (0.01^e24) - (0.13^e34) + (0.51^e1234)